In [ ]:
from qasystem.QASystem import *
from evaluation.xquadevaluation import *
from tqdm.contrib import tzip

In [ ]:
# db = Database()
# db.connect()
# db.delete_all_documents()
df, dicts = xquad_data_prepare('/home/doinakis/github/haystack/xquad-dataset/xquad.el.json')
# db.add_documents(dicts=dicts)
# qa = QASystem(database=db)


In [ ]:
df.answer[:10]

In [ ]:
f1_scores = []
em_scores = []
retriever_em = []
actual_ans = []
predicted_ans = []
for question, answer, doc_id in tzip(df.question, df.answer, df.doc_id):
  prediction = qa.pipe.run(
    query=f"{question}", params={"ESRetriever": {"top_k": 3}, "Reader": {"top_k": 1}}
  )
  actual_ans.append(answer['text'])
  predicted_ans.append(prediction['answers'][0].answer)
  retriever_em.append(int(prediction['documents'][0].meta['name'] == doc_id))
  em = compute_em(prediction['answers'][0].answer, answer['text'])
  em_scores.append(em)
  if em == 1:
    f1_scores.append(1)
  else:
    f1_scores.append(compute_f1(prediction['answers'][0].answer, answer['text']))

scores = pd.DataFrame()
scores['actual'] = actual_ans
scores['predicted'] =predicted_ans
scores['em'] = em_scores
scores['f1'] = f1_scores
scores['retriever_em'] = retriever_em

print(f'Exact Match: {scores.em.mean()}')
print(f'F1-Score: {scores.f1.mean()}')
print(f'Accuracy: {scores.retriever_em.mean()}')